In [12]:
import pandas as pd
import seaborn as sns
import inspect


from src.config import *

    
print('Loading data...\n')
raw_df = pd.read_csv('graphs/constant/raw.csv')


Loading data...



In [5]:
print('Adding graphs...\n')

for server_size in SERVER_SIZES:
    for server_domain in SERVER_DOMAINS:
        for client_sensitivity in CLIENT_SENSITIVITIES:

            # ----------------------------------------------------------------------------------
            # ------------------------------ PROCESS RESULTS -----------------------------------
            # ----------------------------------------------------------------------------------

            # restrict values for graphing in the setting above ^
            partial_df = raw_df.loc[(raw_df['server_size'] == server_size) & (raw_df['server_domain'] == server_domain) & (raw_df['client_sensitivity'] == client_sensitivity)]

            plot_df = pd.DataFrame(columns=[
                # --- experiment settings ---
                'early_stopping_level', 'eps', 'method'

                # --- evaluation metrics ---
                'raw_acc', 'top_5_acc', 'precision', 'recall'
            ])

            # add the values to be plotted to a new df to satisfy seaborn's API format
            for _, row in partial_df.iterrows():
                for nn_method in ['dptt', 'ldp']:

                    value = pd.DataFrame([{
                        # --- experiment settings ---
                        'early_stopping_level': row['early_stopping_level'] - 1, 'eps': row['eps'], 'nn_method': nn_method,

                        # --- evaluation metrics ---
                        'raw_acc': row[f'raw_acc_{nn_method}'], 'top_5_acc': row[f'top_5_acc_{nn_method}'], 'precision': row[f'precision_{nn_method}'], 'recall': row[f'recall_{nn_method}']
                    }])

                    plot_df = pd.concat([plot_df, value], ignore_index=True)


            # ------------------------------------------------------------------------
            # ------------------------------ GRAPH -----------------------------------
            # ------------------------------------------------------------------------

            for metric in ['raw_acc', 'top_5_acc', 'precision', 'recall']: # precision and recall considers top K values relevant

                # plot for both DP-TT and LDP
                plot = sns.lineplot(
                    data=plot_df, x='eps', y=f'{metric}', 
                    hue='early_stopping_level', style='nn_method', markers=True, dashes=['', (1, 1)],
                    palette='deep', alpha=0.75,
                )

                # graph params
                plot.set(
                    ylim=(-0.1, 1.1),
                    title=f'{metric} vs. total epsilon'
                )

                # move legend
                sns.move_legend(plot, "upper left", bbox_to_anchor=(1, 1))

                # save plot
                plot.get_figure().savefig(f'graphs/{metric}.png', bbox_inches='tight', dpi=400)
                plot.get_figure().clf()

Adding graphs...



KeyboardInterrupt: 